In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [3]:
import urllib.request
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop

weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weights_file = "inception_v3.h5"
urllib.request.urlretrieve(weights_url, weights_file)

pre_trained_model = InceptionV3(input_shape=(150, 150, 3),
                                include_top=False,
                                weights=None)

pre_trained_model.load_weights(weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False

pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer=RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['acc'])

training_url = "https://storage.googleapis.com/learning-datasets/horse-or-human.zip"
training_file_name = "horse-or-human.zip"
training_dir = 'horse-or-human/training/'
urllib.request.urlretrieve(training_url, training_file_name)
zip_ref = zipfile.ZipFile(training_file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

validation_url = "https://storage.googleapis.com/learning-datasets/validation-horse-or-human.zip"
validation_file_name = "validation-horse-or-human.zip"
validation_dir = 'horse-or-human/validation/'
urllib.request.urlretrieve(validation_url, validation_file_name)

zip_ref = zipfile.ZipFile(validation_file_name, 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()


# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1.0/255.)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(training_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode='binary',
                                                         target_size=(150, 150))


history = model.fit_generator(
            train_generator,
            validation_data=validation_generator,
            epochs=20,
            verbose=1)


last layer output shape:  (None, 7, 7, 768)
Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


<ipython-input-3-3ea3d2fcc296>:87: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/20
52/52 [==============================] - 24s 232ms/step - loss: 1.1272 - acc: 0.9182 - val_loss: 0.0031 - val_acc: 1.0000
Epoch 2/20
52/52 [==============================] - 11s 207ms/step - loss: 0.0791 - acc: 0.9766 - val_loss: 0.0590 - val_acc: 0.9805
Epoch 3/20
52/52 [==============================] - 11s 203ms/step - loss: 0.0850 - acc: 0.9747 - val_loss: 0.1982 - val_acc: 0.9531
Epoch 4/20
52/52 [==============================] - 11s 205ms/step - loss: 0.0245 - acc: 0.9951 - val_loss: 0.0586 - val_acc: 0.9805
Epoch 5/20
52/52 [==============================] - 11s 209ms/step - loss: 0.0418 - acc: 0.9903 - val_loss: 0.0569 - val_acc: 0.9883
Epoch 6/20
52/52 [==============================] - 11s 202ms/step - loss: 0.0266 - acc: 0.9922 - val_loss: 0.0545 - val_acc: 0.9883
Epoch 7/20
52/52 [==============================] - 11s 203ms/step - loss: 0.0082 - acc: 0.9971 - val_loss: 0.0132 - val_acc: 0.9961
Epoch 8/20
52/52 [==============================] - 10s 201ms/step - 

In [5]:
import numpy as np
from google.colab import files
#from keras.preprocessing import image
import keras.utils as image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  image_tensor = np.vstack([x])
  classes = model.predict(image_tensor)
  print(classes)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")
 

Saving 6d77ed8175e10908144b3af2a48173d9_t.jpeg to 6d77ed8175e10908144b3af2a48173d9_t (1).jpeg
Saving gahag-0037384573-1.jpg to gahag-0037384573-1 (1).jpg
Saving images.jpeg to images (1).jpeg
Saving images22.jpeg to images22 (1).jpeg
Saving images33.jpeg to images33 (1).jpeg
Saving img_os06.jpg to img_os06 (1).jpg
Saving publicdomainq-0032819ktf.jpg to publicdomainq-0032819ktf (1).jpg
1/1 [==============================] - 1s 1s/step
[[1.]]
[1.]
6d77ed8175e10908144b3af2a48173d9_t.jpeg is a human
1/1 [==============================] - 0s 28ms/step
[[1.]]
[1.]
gahag-0037384573-1.jpg is a human
1/1 [==============================] - 0s 29ms/step
[[1.]]
[1.]
images.jpeg is a human
1/1 [==============================] - 0s 27ms/step
[[1.]]
[1.]
images22.jpeg is a human
1/1 [==============================] - 0s 26ms/step
[[1.]]
[1.]
images33.jpeg is a human
1/1 [==============================] - 0s 28ms/step
[[1.]]
[1.]
img_os06.jpg is a human
1/1 [==============================] - 0s 27ms/s